# Create a connection

The process of establishing a connection between two peers, referred to as the Inviter and Invitee, begins with the Inviter creating an out-of-band (oob) invitation. This invitation contains all the necessary information for the Invitee to connect with the Inviter. Once the invitation is created, it must be handed to the Invitee in some way, such as through email, messaging or QR code. Once the Invitee receives the invitation, they must accept it in order to proceed with the connection. Finally, with the invitation accepted, the connection is established, allowing the two peers to communicate and share data.

In [1]:
import os
import datetime
import time
from pprint import pprint
from dotenv import load_dotenv

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.types import Response

### Utilitary functions

In [2]:
def get_invitation_str(connection):
    parts = connection.invitation.invitation_url.split("=")
    return parts[1]

def print_connection(connection):
    print(f"connection_id: {connection.connection_id}")
    print(f"state:         {connection.state}")
    print(f"label:         {connection.label}")
    print(f"my_did:        {connection.my_did}")
    print(f"their_did:     {connection.their_did}")
    print(f"created_at:    {connection.created_at}")
    print(f"OOB Invitation: {get_invitation_str(connection)}")

### Client instances

We will create two separate clients, one for the inviter and one for the invitee, in order to establish a connection between the two. The roles of the inviter and invitee may overlap with the traditional holder, prover, issuer, and verifier relationships. In some cases, the inviter may also have the role of an issuer or verifier, while the invitee may have the role of a holder or prover. This flexibility allows for different scenarios and use cases to be supported within the same flow

note: remember to update the file variables.env with the URLs and API keys provided to you.


In [3]:
load_dotenv("../BetaProgram/variables.env")
inviterApiKey = os.getenv('ISSUER_APIKEY')
inviterUrl = os.getenv('ISSUER_URL')

inviteeApiKey = os.getenv('HOLDER_APIKEY')
inviteeUrl = os.getenv('HOLDER_URL')

inviter_client = Client(base_url=inviterUrl, headers={"apiKey": inviterApiKey})
invitee_client = Client(base_url=inviteeUrl, headers={"apiKey": inviteeApiKey})

### Inviter - Create the invitation

An invitation is created when a connection is created. The only parameter required is a label to identify the connection with a human-readable format. 

The connection state, at the Inviter, will be set to `InvitationGenerated`

In [4]:
conn_request = CreateConnectionRequest()
conn_request.label = f'Connect {datetime.date.today().strftime("%Y-%m-%d %H:%M:%S")}'
inviter_connection: Response[Connection] =  create_connection.sync(client=inviter_client,json_body=conn_request)

invitation = get_invitation_str(inviter_connection)
print_connection(inviter_connection)

connection_id: cb01e66d-6844-4f0a-8c51-7f3486fad705
state:         InvitationGenerated
label:         Connect 2023-02-06 00:00:00
my_did:        <prism_agent_open_api_specification_client.types.Unset object at 0x7ff31b7687c0>
their_did:     <prism_agent_open_api_specification_client.types.Unset object at 0x7ff31b7687c0>
created_at:    2023-02-06 05:42:13.051043+00:00
OOB Invitation: eyJpZCI6ImNiMDFlNjZkLTY4NDQtNGYwYS04YzUxLTdmMzQ4NmZhZDcwNSIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNveXd0b2pxRTh0NlVldmtSOE1VanhLeU5tZUt0cGZhM1l4Mm5GSldMbm5HbS5WejZNa2tjaUs3NVl2NXZjeERFc21jTHoxS3pNNGNaQXlCendRUVVpcE5Yd3g2Zm1BLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92TDJodmMzUXVaRzlqYTJWeUxtbHVkR1Z5Ym1Gc09qZ3dPREF2Wkdsa1kyOXRiU0lzSW5JaU9sdGRMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiLCJib2R5Ijp7ImdvYWxfY29kZSI6ImNvbm5lY3QiLCJnb2FsIjoiRXN0YWJsaXNoIGEgdHJ1c3QgY29ubmVjdGlvbiBiZXR3ZWVuIHR3byBwZWVycyIsImFjY2VwdCI6W119fQ


### Invitee - Accept the invitation 

When the inviter creates the invitation, there is no connection on the invitee side, which is why the invitation is shared out of band. Here we conveniently use a variable to pass the initation to the invitee

The PRISM Agent does the process to establish the connection automatically, so it may not be possible to track all the protocol steps. They progress as described below:

Invitee: ConnectionRequestPending --> ConnectionRequestSent --> ConnectionResponseReceived

Inviter: InvitationGenerated --> ConnectionResponsePending --> ConnectionResponseSent

>**Note -** if the while loop gets stuck replace:  
>```while (inviter_connection.state != 'ConnectionResponseSent' or invitee_connection.state != 'ConnectionResponseReceived'):```    
>    with:  
>```while (inviter_connection.state != 'ConnectionResponseSent' or not(invitee_connection.state == 'ConnectionResponseReceived' or invitee_connection.state == 'ConnectionRequestSent')):```  
>and raise a ZenDesk Ticket :)

In [5]:
print("Please wait...")

accept_conn_request = AcceptConnectionInvitationRequest(invitation)
invitee_connection: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=invitee_client,json_body=accept_conn_request)

while (inviter_connection.state != 'ConnectionResponseSent' or invitee_connection.state != 'ConnectionResponseReceived'):
    inviter_connection: Response[Connection] = get_connection.sync(client=inviter_client,connection_id=inviter_connection.connection_id)
    invitee_connection: Response[Connection] = get_connection.sync(client=invitee_client,connection_id=invitee_connection.connection_id)
    print("Inviter State: {} / Invitee State: {} \n".format(inviter_connection.state,invitee_connection.state))
    time.sleep(1)
    
print("Connection established between Issuer and Holder!")

Please wait...
Inviter State: InvitationGenerated / Invitee State: ConnectionRequestPending 

Inviter State: InvitationGenerated / Invitee State: ConnectionRequestPending 

Inviter State: InvitationGenerated / Invitee State: ConnectionRequestPending 

Inviter State: InvitationGenerated / Invitee State: ConnectionRequestPending 

Inviter State: ConnectionResponsePending / Invitee State: ConnectionRequestPending 

Inviter State: ConnectionResponsePending / Invitee State: ConnectionRequestSent 

Inviter State: ConnectionResponsePending / Invitee State: ConnectionRequestSent 

Inviter State: ConnectionResponsePending / Invitee State: ConnectionRequestSent 

Inviter State: ConnectionResponsePending / Invitee State: ConnectionRequestSent 

Inviter State: ConnectionResponsePending / Invitee State: ConnectionRequestSent 

Inviter State: ConnectionResponsePending / Invitee State: ConnectionRequestSent 

Inviter State: ConnectionResponsePending / Invitee State: ConnectionRequestSent 

Inviter St

### Inviter - Check connection

The details of the connection on the Inviter side are as presented below:

In [6]:
inviter_connection: Response[Connection] =  get_connection.sync(client=inviter_client,connection_id=inviter_connection.connection_id)
print_connection(inviter_connection)

connection_id: cb01e66d-6844-4f0a-8c51-7f3486fad705
state:         ConnectionResponseSent
label:         Connect 2023-02-06 00:00:00
my_did:        did:peer:2.Ez6LSoywtojqE8t6UevkR8MUjxKyNmeKtpfa3Yx2nFJWLnnGm.Vz6MkkciK75Yv5vcxDEsmcLz1KzM4cZAyBzwQQUipNXwx6fmA.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0
their_did:     did:peer:2.Ez6LSkNU7c9yUMKFP2YEAwaqro2uhmoHtcMQQp5N2uxhNDwCU.Vz6MkhqYzHqdMiQsSiQ1tnbZVMpKrRBhto8koCoudgYppQiDR.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0
created_at:    2023-02-06 05:42:13+00:00
OOB Invitation: eyJpZCI6ImNiMDFlNjZkLTY4NDQtNGYwYS04YzUxLTdmMzQ4NmZhZDcwNSIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNveXd0b2pxRTh0NlVldmtSOE1VanhLeU5tZUt0cGZhM1l4Mm5GSldMbm5HbS5WejZNa2tjaUs3NVl2NXZjeERFc21jTHoxS3pNNGNaQXlCendRUVVpcE5Yd3g2Zm1BLlNleUowSWpvaVpHMGlMQ0p6SWpvaWFIUjBjRG92

### Invitee - Check connection

The details of the connection on the Invitee side are as presented below:

In [7]:
invitee_connection: Response[Connection] =  get_connection.sync(client=invitee_client,connection_id=invitee_connection.connection_id)
print_connection(invitee_connection)

connection_id: f6f88329-1eb4-44f1-bc67-4f597bfd1d66
state:         ConnectionResponseReceived
label:         <prism_agent_open_api_specification_client.types.Unset object at 0x7ff31b7687c0>
my_did:        did:peer:2.Ez6LSkNU7c9yUMKFP2YEAwaqro2uhmoHtcMQQp5N2uxhNDwCU.Vz6MkhqYzHqdMiQsSiQ1tnbZVMpKrRBhto8koCoudgYppQiDR.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0
their_did:     did:peer:2.Ez6LSoywtojqE8t6UevkR8MUjxKyNmeKtpfa3Yx2nFJWLnnGm.Vz6MkkciK75Yv5vcxDEsmcLz1KzM4cZAyBzwQQUipNXwx6fmA.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0
created_at:    2023-02-06 05:42:14+00:00
OOB Invitation: eyJpZCI6ImNiMDFlNjZkLTY4NDQtNGYwYS04YzUxLTdmMzQ4NmZhZDcwNSIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNveXd0b2pxRTh0NlVldmtSOE1VanhLeU5tZUt0cGZhM1l4Mm5GSldMbm5HbS5WejZNa2tjaUs3NVl2NXZjeERFc21jTHoxS3pNNGNaQXlCend

### List all connections

The following code retrieves the lists of connections. (only printing the first 3)

In [8]:
inviter_connections: Response[ConnectionCollection] = get_connections.sync(client=inviter_client)
invitee_connections: Response[ConnectionCollection] = get_connections.sync(client=invitee_client)

print("Inviter connections")
print("-------------------\n")
pprint(f"{inviter_connections.contents[0:3]}")
print("\nInvitee connections")
print("-------------------\n")
pprint(f"{invitee_connections.contents[0:3]}")

Inviter connections
-------------------

("[Connection(self_='Connection', "
 "kind='/connections/cb01e66d-6844-4f0a-8c51-7f3486fad705', "
 "connection_id='cb01e66d-6844-4f0a-8c51-7f3486fad705', "
 'state=<ConnectionAllOfState.CONNECTION_RESPONSE_SENT: '
 "'ConnectionResponseSent'>, created_at=datetime.datetime(2023, 2, 6, 5, 42, "
 '13, tzinfo=tzutc()), '
 "invitation=ConnectionInvitation(id='cb01e66d-6844-4f0a-8c51-7f3486fad705', "
 "type='https://didcomm.org/out-of-band/2.0/invitation', "
 "from_='did:peer:2.Ez6LSoywtojqE8t6UevkR8MUjxKyNmeKtpfa3Yx2nFJWLnnGm.Vz6MkkciK75Yv5vcxDEsmcLz1KzM4cZAyBzwQQUipNXwx6fmA.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0', "
 "invitation_url='https://domain.com/path?_oob=eyJpZCI6ImNiMDFlNjZkLTY4NDQtNGYwYS04YzUxLTdmMzQ4NmZhZDcwNSIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNveXd0b2pxRTh0NlVldmtSOE1VanhLeU5tZUt0cGZhM1l4Mm5GSld